In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('laptop_data.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [3]:
df['Ram'] = df['Ram'].str.replace('GB','').astype('int32')
df['Weight'] = df['Weight'].str.replace('kg','').astype('float32')

In [4]:
df['Touchscreen'] = df['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0)

In [5]:
df['Ips'] = df['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0)

In [6]:
new = df['ScreenResolution'].str.split('x',n=1,expand=True)
df['X_res'] = new[0]
df['Y_res'] = new[1]
df['X_res'] = df['X_res'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x:x[0])

In [7]:
df['X_res'] = df['X_res'].astype('int')
df['Y_res'] = df['Y_res'].astype('int')
df['ppi'] = (((df['X_res']**2) + (df['Y_res']**2))**0.5/df['Inches']).astype('float')
df.drop(columns=['ScreenResolution'],inplace=True)

In [8]:
df.drop(columns=['Inches','X_res','Y_res'],inplace=True)
df['Cpu']

0                       Intel Core i5 2.3GHz
1                       Intel Core i5 1.8GHz
2                 Intel Core i5 7200U 2.5GHz
3                       Intel Core i7 2.7GHz
4                       Intel Core i5 3.1GHz
                        ...                 
1298              Intel Core i7 6500U 2.5GHz
1299              Intel Core i7 6500U 2.5GHz
1300    Intel Celeron Dual Core N3050 1.6GHz
1301              Intel Core i7 6500U 2.5GHz
1302    Intel Celeron Dual Core N3050 1.6GHz
Name: Cpu, Length: 1303, dtype: object

In [9]:
df['Cpu Name'] = df['Cpu'].apply(lambda x:" ".join(x.split()[0:3]))

In [10]:
def fetch_processor(text):
    if text == 'Intel Core i7' or text == 'Intel Core i5' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'

df['Cpu brand'] = df['Cpu Name'].apply(fetch_processor)
df['Cpu brand']

0               Intel Core i5
1               Intel Core i5
2               Intel Core i5
3               Intel Core i7
4               Intel Core i5
                ...          
1298            Intel Core i7
1299            Intel Core i7
1300    Other Intel Processor
1301            Intel Core i7
1302    Other Intel Processor
Name: Cpu brand, Length: 1303, dtype: object

In [11]:
df.drop(columns=['Cpu','Cpu Name'],inplace=True)
df['Memory']

0                 128GB SSD
1       128GB Flash Storage
2                 256GB SSD
3                 512GB SSD
4                 256GB SSD
               ...         
1298              128GB SSD
1299              512GB SSD
1300     64GB Flash Storage
1301                1TB HDD
1302              500GB HDD
Name: Memory, Length: 1303, dtype: object

In [12]:
df.sample(3)

,Company,TypeName,Ram,Memory,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand
619,Acer,2 in 1 Convertible,4,32GB Flash Storage,Intel HD Graphics 400,Chrome OS,1.25,20193.12,1,1,135.094211,Other Intel Processor
283,Lenovo,Notebook,6,256GB SSD,Intel HD Graphics 620,Windows 10,2.20,30849.12,0,0,141.211998,Intel Core i5
370,Asus,2 in 1 Convertible,12,512GB SSD + 2TB HDD,Nvidia GeForce GT 940MX,Windows 10,2.26,58554.72,1,0,141.211998,Intel Core i7


In [13]:
df['mem']=df['Memory'].str.split(' ')
def fetch_processor(text):
    types = []
    if 'SSD' in text:
        types.append('SSD')
    if 'HDD' in text:
        types.append('HDD')
    if 'Flash' in text:
        types.append('Flash Storage')
    if 'Hybrid' in text:
        types.append('Hybrid')
    return ' '.join(types)

df['_Memory'] = df['Memory'].apply(fetch_processor)
df

,Company,TypeName,Ram,Memory,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand,mem,_Memory
0,Apple,Ultrabook,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,71378.6832,0,1,226.983005,Intel Core i5,"[128GB, SSD]",SSD
1,Apple,Ultrabook,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,47895.5232,0,0,127.677940,Intel Core i5,"[128GB, Flash, Storage]",Flash Storage
2,HP,Notebook,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,30636.0000,0,0,141.211998,Intel Core i5,"[256GB, SSD]",SSD
3,Apple,Ultrabook,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,135195.3360,0,1,220.534624,Intel Core i7,"[512GB, SSD]",SSD
4,Apple,Ultrabook,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,96095.8080,0,1,226.983005,Intel Core i5,"[256GB, SSD]",SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,4,128GB SSD,Intel HD Graphics 520,Windows 10,1.80,33992.6400,1,1,157.350512,Intel Core i7,"[128GB, SSD]",SSD
1299,Lenovo,2 in 1 Convertible,16,512GB SSD,Intel HD Graphics 520,Windows 10,1.30,79866.7200,1,1,276.053530,Intel Core i7,"[512GB, SSD]",SSD
1300,Lenovo,Notebook,2,64GB Flash Storage,Intel HD Graphics,Windows 10,1.50,12201.1200,0,0,111.935204,Other Intel Processor,"[64GB, Flash, Storage]",Flash Storage
1301,HP,Notebook,6,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19,40705.9200,0,0,100.454670,Intel Core i7,"[1TB, HDD]",HDD


In [14]:
# df["Layer1HDD"] = df["_Memory"].apply(lambda x: 1 if "HDD" in x else 0)
# df["Layer1SSD"] = df["_Memory"].apply(lambda x: 1 if "SSD" in x else 0)
# df["Layer1Hybrid"] = df["_Memory"].apply(lambda x: 1 if "Hybrid" in x else 0)
# df["Layer1Flash_Storage"] = df["_Memory"].apply(lambda x: 1 if "Flash Storage" in x else 0)

In [15]:
df

,Company,TypeName,Ram,Memory,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand,mem,_Memory
0,Apple,Ultrabook,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,71378.6832,0,1,226.983005,Intel Core i5,"[128GB, SSD]",SSD
1,Apple,Ultrabook,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,47895.5232,0,0,127.677940,Intel Core i5,"[128GB, Flash, Storage]",Flash Storage
2,HP,Notebook,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,30636.0000,0,0,141.211998,Intel Core i5,"[256GB, SSD]",SSD
3,Apple,Ultrabook,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,135195.3360,0,1,220.534624,Intel Core i7,"[512GB, SSD]",SSD
4,Apple,Ultrabook,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,96095.8080,0,1,226.983005,Intel Core i5,"[256GB, SSD]",SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,4,128GB SSD,Intel HD Graphics 520,Windows 10,1.80,33992.6400,1,1,157.350512,Intel Core i7,"[128GB, SSD]",SSD
1299,Lenovo,2 in 1 Convertible,16,512GB SSD,Intel HD Graphics 520,Windows 10,1.30,79866.7200,1,1,276.053530,Intel Core i7,"[512GB, SSD]",SSD
1300,Lenovo,Notebook,2,64GB Flash Storage,Intel HD Graphics,Windows 10,1.50,12201.1200,0,0,111.935204,Other Intel Processor,"[64GB, Flash, Storage]",Flash Storage
1301,HP,Notebook,6,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19,40705.9200,0,0,100.454670,Intel Core i7,"[1TB, HDD]",HDD


In [16]:
def extract_storage(memory_str, storage_type):
    pattern = re.compile(r'(\d+)(GB|TB) ' + storage_type)
    matches = pattern.findall(memory_str)
    total_storage = 0
    for match in matches:
        size, unit = match
        size = int(size)
        if unit == 'TB':
            size *= 1024  # Convert TB to GB
        total_storage += size
    return total_storage

# Extract SSD, HDD, Flash Storage, and Hybrid Storage amounts
df['SSD'] = df['Memory'].apply(lambda x: extract_storage(x, 'SSD'))
df['HDD'] = df['Memory'].apply(lambda x: extract_storage(x, 'HDD'))
df['Flash Storage'] = df['Memory'].apply(lambda x: extract_storage(x, 'Flash Storage'))
df['Hybrid'] = df['Memory'].apply(lambda x: extract_storage(x, 'Hybrid'))


In [17]:
df.sample(1)

,Company,TypeName,Ram,Memory,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand,mem,_Memory,SSD,HDD,Flash Storage,Hybrid
906,HP,Netbook,8,512GB SSD,Intel HD Graphics 620,Windows 10,1.26,79014.24,0,0,176.232574,Intel Core i7,"[512GB, SSD]",SSD,512,0,0,0


In [18]:
df.drop(columns=['Memory','mem','_Memory'],inplace=True)
df.sample(1)

,Company,TypeName,Ram,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand,SSD,HDD,Flash Storage,Hybrid
390,Asus,Gaming,8,Nvidia GeForce GTX 1050,Windows 10,3.0,55357.92,0,0,127.335675,Intel Core i7,0,1024,0,0


In [19]:
df.drop(columns=['Hybrid','Flash Storage'],inplace=True)
df.sample(1)

,Company,TypeName,Ram,Gpu,OpSys,Weight,Price,Touchscreen,Ips,ppi,Cpu brand,SSD,HDD
666,Dell,Notebook,8,Intel HD Graphics 620,Windows 10,2.06,50882.4,0,0,141.211998,Intel Core i5,0,1024


In [20]:
df['Gpu brand'] = df['Gpu'].apply(lambda x:x.split()[0])

In [21]:
df['Gpu brand'].value_counts()

Gpu brand
Intel     722
Nvidia    400
AMD       180
ARM         1
Name: count, dtype: int64

In [22]:
df = df[df['Gpu brand'] != 'ARM']

In [23]:
df.drop(columns=['Gpu'],inplace=True)

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_10264\1111925144.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Gpu'],inplace=True)


In [24]:
def cat_os(inp):
    if inp == 'Windows 10' or inp == 'Windows 7' or inp == 'Windows 10 S':
        return 'Windows'
    elif inp == 'macOS' or inp == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others/No OS/Linux'

df['os'] = df['OpSys'].apply(cat_os)
df.drop(columns=['OpSys'],inplace=True)

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_10264\2248303200.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['os'] = df['OpSys'].apply(cat_os)
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_10264\2248303200.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['OpSys'],inplace=True)


In [25]:
X = df.drop(columns=['Price'])
y = np.log(df['Price'])

In [26]:
X

,Company,TypeName,Ram,Weight,Touchscreen,Ips,ppi,Cpu brand,SSD,HDD,Gpu brand,os
0,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,128,0,Intel,Mac
1,Apple,Ultrabook,8,1.34,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,0,0,141.211998,Intel Core i5,256,0,Intel,Others/No OS/Linux
3,Apple,Ultrabook,16,1.83,0,1,220.534624,Intel Core i7,512,0,AMD,Mac
4,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,256,0,Intel,Mac
...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,4,1.80,1,1,157.350512,Intel Core i7,128,0,Intel,Windows
1299,Lenovo,2 in 1 Convertible,16,1.30,1,1,276.053530,Intel Core i7,512,0,Intel,Windows
1300,Lenovo,Notebook,2,1.50,0,0,111.935204,Other Intel Processor,0,0,Intel,Windows
1301,HP,Notebook,6,2.19,0,0,100.454670,Intel Core i7,0,1024,AMD,Windows


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

## Random Forest

In [29]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse_output=False,drop='first'),[0,1,7,10,11])
],remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,
                              random_state=3,
                              max_samples=0.5,
                              max_features=0.75,
                              max_depth=15)

pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred)*100)
print('MAE',mean_absolute_error(y_test,y_pred)*100)

R2 score 88.31292867151932
MAE 15.995313001529835


## Voting Regressor

## Exporting The Model

In [30]:
import pickle

pickle.dump(df,open('df.pkl','wb'))
pickle.dump(pipe,open('pipe.pkl','wb'))